In [69]:
import streamlit as st
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [70]:
def load_data():
    file = 'aircrahesFullDataUpdated_2024.csv'
    df = pd.read_csv(file)

    return df

In [71]:
df = load_data()

In [72]:
df.head()

,Year,Quarter,Month,Day,Country/Region,Aircraft Manufacturer,Aircraft,Location,Operator,Ground,Fatalities (air),Aboard
0,1970,Qtr 2,April,28,NaN,de Havilland Canada,de Havilland Canada DHC 6 Twin Otter 100VH,Kainatu,Airlines Australia GuineaTrans New,0,8,11
1,1952,Qtr 2,April,30,NaN,Douglas,Douglas DC 3VT,Delhi,Airways IndiaDeccan,0,9,9
2,1973,Qtr 2,April,10,NaN,Vickers Vanguard,Vickers Vanguard 952G AXOP,Near Basel,(UK) Airlines International SwitzerlandInvicta,0,108,145
3,2000,Qtr 2,April,19,'-,Antonov,Antonov AN 8TL,Pepa -,Airlines Centrafricain,0,24,24
4,1946,Qtr 2,April,30,'-,Douglas,Douglas C 47A FZ559,Pindi-Khut -,Force Air Royal,0,12,12


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5035 entries, 0 to 5034
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Year                   5035 non-null   int64 
 1   Quarter                5035 non-null   object
 2   Month                  5035 non-null   object
 3   Day                    5035 non-null   int64 
 4   Country/Region         4805 non-null   object
 5   Aircraft Manufacturer  5035 non-null   object
 6   Aircraft               5035 non-null   object
 7   Location               5035 non-null   object
 8   Operator               4801 non-null   object
 9   Ground                 5035 non-null   int64 
 10  Fatalities (air)       5035 non-null   int64 
 11  Aboard                 5035 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 472.2+ KB


In [74]:
# Step 1:  Handling Missing Values and Standardizing Entries

# Replace '-' with NaN in 'Country/Region'
df['Country/Region'] = df['Country/Region'].replace('-', pd.NA)

# Fill missing values in 'Country/Region' and 'Operator' with 'Unknown'
df['Country/Region'] = df['Country/Region'].fillna('Unknown')
df['Operator'] = df['Operator'].fillna('Unknown')

In [75]:
# Step 2: Trim and Clean Text Fields
# Columns to clean: 'Country/Region', 'Aircraft', 'Location', 'Operator'
df['Country/Region'] = df['Country/Region'].str.strip().str.title()
df['Aircraft Manufacturer'] = df['Aircraft Manufacturer'].str.strip().str.title()
df['Aircraft'] = df['Aircraft'].str.strip()
df['Location'] = df['Location'].str.strip()
df['Operator'] = df['Operator'].str.strip()

In [76]:
# Step 3: Convert 'Quarter' and 'Month' to categorical data types
df['Quarter'] = pd.Categorical(df['Quarter'], categories=['Qtr 1', 'Qtr 2', 'Qtr 3', 'Qtr 4'])
df['Month'] = pd.Categorical(df['Month'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'])

In [77]:
# Step 4: Handling Outliers
# Calculate the interquartile range (IQR) for 'Ground' and 'Fatalities (air)' to detect outliers
Q1_ground = df['Ground'].quantile(0.25)
Q3_ground = df['Ground'].quantile(0.75)
IQR_ground = Q3_ground - Q1_ground

Q1_fatalities = df['Fatalities (air)'].quantile(0.25)
Q3_fatalities = df['Fatalities (air)'].quantile(0.75)
IQR_fatalities = Q3_fatalities - Q1_fatalities

# Define outlier bounds
lower_bound_ground = Q1_ground - 1.5 * IQR_ground
upper_bound_ground = Q3_ground + 1.5 * IQR_ground

lower_bound_fatalities = Q1_fatalities - 1.5 * IQR_fatalities
upper_bound_fatalities = Q3_fatalities + 1.5 * IQR_fatalities

# Cap outliers in 'Ground' and 'Fatalities (air)'
df['Ground'] = df['Ground'].clip(lower=lower_bound_ground, upper=upper_bound_ground)
df['Fatalities (air)'] = df['Fatalities (air)'].clip(lower=lower_bound_fatalities, upper=upper_bound_fatalities)

In [78]:
# Step 5: Check for Duplicates
duplicate_rows = df.duplicated().sum()

# If any duplicates are found, remove them
df.drop_duplicates(inplace=True)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5035 entries, 0 to 5034
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Year                   5035 non-null   int64   
 1   Quarter                5035 non-null   category
 2   Month                  5035 non-null   category
 3   Day                    5035 non-null   int64   
 4   Country/Region         5035 non-null   object  
 5   Aircraft Manufacturer  5035 non-null   object  
 6   Aircraft               5035 non-null   object  
 7   Location               5035 non-null   object  
 8   Operator               5035 non-null   object  
 9   Ground                 5035 non-null   int64   
 10  Fatalities (air)       5035 non-null   float64 
 11  Aboard                 5035 non-null   int64   
dtypes: category(2), float64(1), int64(4), object(5)
memory usage: 403.9+ KB


# RESEARCH QUESTION
1. How has the number of air crashes evolved over time:
This question examines the trend in the frequency of air crashes over various years. By analyzing yearly crash data, we can identify whether crashes have increased or decreased over time, helping to understand patterns or impacts of safety improvements.
2. Which aircrafts has experienced the highest number of crashes
This question focuses on identifying aircraft models or manufacturers that have had the highest number of crashes. By analyzing the data, we can pinpoint which aircraft are involved in more crashes, which may indicate potential design or operational issues.
3.  Aircraft and Operator Analysis:
Which aircraft manufacturers are most frequently involved in crashes?
Feature: A bar chart showing the top 10 aircraft manufacturers with the most crashes.
Importance: Identifying which manufacturers have higher crash rates can be crucial for understanding potential safety concerns related to specific aircraft types.
4. Safety and Prevention:
Has aircraft safety improved over time based on fatality rates?
Feature: A line chart comparing the number of fatalities over time.
Importance: This analysis can show whether improvements in aviation safety technology and regulations have effectively reduced fatality rates.
  

In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st

# Load dataset
df = pd.read_csv('aircrahesFullDataUpdated_2024.csv')
df = load_data()

# Title of the app
st.title('Aircraft Crashes Data Explorer')



# Group by year and count crashes
accidents_per_year = df['Year'].value_counts().sort_index()

# Plot in Streamlit
st.subheader('Air Crashes Over Time')
st.line_chart(accidents_per_year)




DeltaGenerator()

In [82]:
# Count crashes by airline
airline_crashes = df['Airline'].value_counts().head(10)

# Plot in Streamlit
st.subheader('Top 10 Airlines with Highest Number of Crashes')
st.bar_chart(airline_crashes)




DeltaGenerator()

In [91]:
import matplotlib.pyplot as plt
import pandas as pd
import streamlit as st

# Assuming the dataset is already loaded into df
st.header('Top 10 Aircraft Manufacturers Involved in Crashes')

# Get the top 10 aircraft manufacturers involved in crashes
top_manufacturers = df['Aircraft Manufacturer'].value_counts().head(10)

# Plotting the bar chart
plt.figure(figsize=(10, 6))
top_manufacturers.plot(kind='bar', color='orange')
plt.title('Top 10 Aircraft Manufacturers Involved in Crashes')
plt.xlabel('Aircraft Manufacturer')
plt.ylabel('Number of Crashes')

# Show plot in Streamlit
st.pyplot(plt)


DeltaGenerator()

In [84]:
# 5. Fatalities Over Time
st.header('Fatalities Over Time')
fatalities_per_year = df.groupby('Year')['Fatalities (air)'].sum()
plt.figure(figsize=(10, 6))
plt.plot(fatalities_per_year.index, fatalities_per_year.values, marker='o', linestyle='-', color='r')
plt.title('Total Fatalities per Year')
plt.xlabel('Year')
plt.ylabel('Total Fatalities')
st.pyplot(plt)

# Option to download the filtered data
st.header('Download the filtered data')
st.write("You can download the filtered dataset as a CSV file.")
csv = df.to_csv(index=False).encode('utf-8')
st.download_button(label="Download CSV", data=csv, file_name='filtered_aircrashes_data.csv', mime='text/csv')

False